In [8]:
%%file run_2thre_one.py

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from scipy.stats.mstats import winsorize
from random import seed
from random import random
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


def run_2thre_one(q, n_fee = 3, consider_funding_rate = False):
    def load_data(load):
        spot = pd.read_csv(f'data/spot_{load}_usdt_1h.csv', parse_dates=['timestamp'])
        perp = pd.read_csv(f'data/perp_{load}_usdt_1h.csv', parse_dates=['timestamp'])
        perp.sort_values(by = 'epoch', ascending=True, inplace=True)
        spot.sort_values(by = 'epoch', ascending=True, inplace=True)
        perp = perp.reset_index(drop=True)
        spot = spot.reset_index(drop=True)
        funding_rate = pd.read_csv(f'data/rate_{load}_usdt.csv', parse_dates=['timestamp'])
        perp['close'] = perp['close'].shift(1)
        perp = perp.dropna(subset = ['close']).reset_index(drop = True)
        spot['close'] = spot['close'].shift(1)
        spot = spot.dropna(subset = ['close']).reset_index(drop = True)
        return spot, perp, funding_rate

    def create_panel(spot, perp, funding_rate):
        panel = perp[['epoch', 'timestamp', 'close', 'usd_volume']]
        panel['day'] = panel['timestamp'].dt.round('D')
        panel = panel.rename(columns={'close': 'perp', 'usd_volume': 'perp_usd_volume'})
        panel = pd.merge(panel, spot[['epoch', 'close', 'usd_volume']], on='epoch', how='left')
        panel = panel.rename(columns={'close': 'spot', 'usd_volume': 'spot_usd_volume'})
        panel = pd.merge(panel, funding_rate[['epoch', 'funding_rate']], on='epoch', how='left')
        panel.sort_values(by = 'epoch', ascending=True, inplace=True)
        panel = panel.reset_index(drop=True)
        return panel

    def open_long_position(ethusdt_spot, ethusdt_perp, cash_acc):
        position_spot = share_spot*(cash_acc/ethusdt_spot)*(1-fee_spot)
        position_perp = share_perp*(cash_acc/ethusdt_perp)*(1-fee_perp)*-1
        fee = cash_acc*(share_spot*fee_spot+share_perp*fee_perp)
        cash_acc = 0

        return position_spot, position_perp, cash_acc, fee

    def close_long_position(position_spot, position_perp, ethusdt_spot, ethusdt_perp, cash_acc):
        spot_value = position_spot*ethusdt_spot
        perp_value = (position_perp*-1)*(buyin_perp*2-ethusdt_perp)

        fee = spot_value*fee_spot+perp_value*fee_perp
        cash_acc = cash_acc+spot_value*(1-fee_spot)+perp_value*(1-fee_perp)

        position_spot = 0
        position_perp = 0

        return position_spot, position_perp, cash_acc, fee

    def open_short_position(ethusdt_spot, ethusdt_perp, cash_acc):
        position_spot = share_spot*(cash_acc/ethusdt_spot)*(1-fee_spot)*-1
        position_perp = share_perp*(cash_acc/ethusdt_perp)*(1-fee_perp)
        fee = cash_acc*(share_spot*fee_spot+share_perp*fee_perp)
        cash_acc = 0

        return position_spot, position_perp, cash_acc, fee

    def close_short_position(position_spot, position_perp, ethusdt_spot, ethusdt_perp, cash_acc):
        spot_value = (position_spot*-1)*(buyin_spot*2-ethusdt_spot)
        perp_value = position_perp*ethusdt_perp

        fee = spot_value*fee_spot+perp_value*fee_perp
        cash_acc = cash_acc+spot_value*(1-fee_spot)+perp_value*(1-fee_perp)

        position_spot = 0
        position_perp = 0

        return position_spot, position_perp, cash_acc, fee

    def calculate_equity(position_spot, position_perp, cash_acc, equity):
        equity_old = equity

        if spot == 'long':
            spot_value = position_spot*ethusdt_spot
            perp_value = (position_perp*-1)*(buyin_perp*2-ethusdt_perp)

        elif spot == 'short':
            spot_value = (position_spot*-1)*(buyin_spot*2-ethusdt_spot)
            perp_value = position_perp*ethusdt_perp

        else: #no position
            spot_value = 0.0
            perp_value = 0.0

        equity = cash_acc + spot_value + perp_value
        pnl = equity - equity_old
        return equity, pnl, spot_value, perp_value

    def calculate_payment(position_perp, ethusdt_perp, funding_rate):
        fr_facevalue = position_perp*ethusdt_perp*-1 #*-1 -> if funding_rate < 0 short perps, pay long perps
        payment = fr_facevalue * funding_rate
        return payment
    
    currencies = ['btc', 'eth', 'bnb', 'doge', 'ada']
    string = ['no', 'low', 'medium-low', 'medium-high']
    fee_spot_lst = [0.0, 0.00015, 0.000375, 0.000525]
    fee_perp_lst = [0.0, 0.0, 0.000054, 0.000108]

    fee_spot = fee_spot_lst[n_fee-1]
    fee_perp = fee_perp_lst[n_fee-1]

    threshold_lst = list(np.arange(.0,1.0,0.05))
    thre_all = []
    for i in threshold_lst:
        for j in threshold_lst:
            if i <= j:
                thre_all.append((j, i))

    thre = thre_all[q-1]
    threshold = thre[0]
    threshold2 = thre[1]

    notional = 1000000.0
    plot = False
    wins = False

    panel_all = []
    for i in range (0,len(currencies)):
        spot, perp, funding_rate = load_data(currencies[i])
        panel_temp = create_panel(spot, perp, funding_rate)
        panel_all.append(panel_temp)
    for i in range (0,len(currencies)):
        T = 1/(3*365)
        panel_all[i]['cip'] = -(1/T)*(np.log(panel_all[i]['perp']) - np.log(panel_all[i]['spot'])) 
        if wins:
            panel_all[i]['cip'] = winsorize(panel_all[i]['cip'], limits = (0.025, 0.025))
        mean = np.round(panel_all[i]['cip'].mean(),4)
        days = (panel_all[i]['day'].iloc[-1] - panel_all[i]['day'].iloc[0]).days

    cutoff = '2022-11-14 00:00:00'

    eth_start = '2019-11-27 07:00:00'
    btc_start = '2019-09-10 08:00:00'
    doge_start = '2020-07-10 09:00:00'
    bnb_start = '2020-02-10 08:00:00'
    ada_start = '2020-01-31 08:00:00'

    for i in range (0,len(currencies)):
        panel_all[i] = panel_all[i][panel_all[i]['timestamp'] <= cutoff] #cutoff, such that all data is equally long

    for i in range (0,len(currencies)):

        panel_all[i]['spot'] = panel_all[i]['spot'].fillna(method='ffill')
        panel_all[i]['spot_usd_volume'] = panel_all[i]['spot_usd_volume'].fillna(method='ffill')
        panel_all[i]['funding_rate'] = panel_all[i]['funding_rate'].fillna(0)
        if consider_funding_rate:
            panel_all[i]['cip'] = panel_all[i]['cip'] - panel_all[i]['funding_rate'] * 365 * 3
        panel_all[i]['iusdt'] = 0
        #panel_all[i] = panel_all[i].dropna()
        panel_all[i].sort_values(by = 'epoch', ascending=True, inplace=True)
        panel_all[i] = panel_all[i].reset_index(drop=True)

    share_spot = (1-fee_perp)/(2-fee_spot-fee_perp)
    share_perp = 1-share_spot

    df_all = []

    for t in range(0, len(currencies)):
        panel = panel_all[t]
        #starting variables
        lst=[]
        position_open = False
        position_spot = 0.0
        position_perp = 0.0
        cash_acc = notional

        spot = 'none'

        equity = cash_acc
        turnover = 0.0
        buyin_spot = 0.0
        buyin_perp = 0.0


        for i in range(0, len(panel)):
            action = False

            cip = panel['cip'][i]
            ethusdt_spot = panel['spot'][i]
            ethusdt_perp = panel['perp'][i]
            fee = 0.0

            if cip < -threshold: #cip smaller than threshold; short perp, long spot            

                if position_open == False:
                    turnover = turnover + cash_acc
                    position_spot, position_perp, cash_acc, fee = open_long_position(ethusdt_spot, ethusdt_perp, cash_acc)
                    buyin_spot = ethusdt_spot
                    buyin_perp = ethusdt_perp

                    position_open = True
                    action = True
                    spot = 'long'

                elif position_open == True:
                    if spot == 'long':
                        pass      

                    elif spot == 'short':
                        position_spot, position_perp, cash_acc, fee_temp1 = close_short_position(position_spot, position_perp, ethusdt_spot, ethusdt_perp, cash_acc)
                        turnover = turnover + cash_acc*2
                        position_spot, position_perp, cash_acc, fee_temp2 = open_long_position(ethusdt_spot, ethusdt_perp, cash_acc)
                        buyin_spot = ethusdt_spot
                        buyin_perp = ethusdt_perp

                        fee = fee_temp1 + fee_temp2

                        action = True
                        spot = 'long'


            elif (cip > -threshold2): #cip is within no-action zone       
                if position_open == False:
                    pass

                elif position_open == True:
                    if spot == 'long':
                        position_spot, position_perp, cash_acc, fee = close_long_position(position_spot, position_perp, ethusdt_spot, ethusdt_perp, cash_acc)
                        turnover = turnover + cash_acc

                    elif spot == 'short':
                        position_spot, position_perp, cash_acc, fee = close_short_position(position_spot, position_perp, ethusdt_spot, ethusdt_perp, cash_acc)
                        turnover = turnover + cash_acc

                    buyin_spot = 0.0
                    buyin_perp = 0.0

                    position_open = False
                    action = True
                    spot = 'none'

            elif (cip >= -threshold) & (cip <= -threshold2):
                if position_open == False:
                    pass

                elif position_open == True:
                    if spot == 'long':
                        pass

                    elif spot == 'short':
                        position_spot, position_perp, cash_acc, fee_temp1 = close_short_position(position_spot, position_perp, ethusdt_spot, ethusdt_perp, cash_acc)
                        turnover = turnover + cash_acc*2
                        position_spot, position_perp, cash_acc, fee_temp2 = open_long_position(ethusdt_spot, ethusdt_perp, cash_acc)
                        buyin_spot = ethusdt_spot
                        buyin_perp = ethusdt_perp

                        fee = fee_temp1 + fee_temp2

                        action = True
                        spot = 'long'

            payment = calculate_payment(position_perp, ethusdt_perp, panel['funding_rate'][i])
            cash_acc = cash_acc + payment
            equity, pnl, spot_value, perp_value = calculate_equity(position_spot, position_perp, cash_acc, equity)
            lst.append([panel['epoch'][i], panel['timestamp'][i], position_open, cash_acc, equity, payment, position_spot, position_perp, fee, cip, panel['funding_rate'][i],  buyin_spot, buyin_perp, ethusdt_spot, ethusdt_perp, pnl, action, turnover, spot, spot_value, perp_value])


        df = pd.DataFrame(lst, columns=['epoch', 'timestamp', 'position_open', 'cash_acc', 'equity', 'funding_payment', 'position_spot', 'position_perp', 'fee', 'cip', 'funding_rate', 'buyin_spot', 'buyin_perp', 'ethusdt_spot', 'ethusdt_perp', 'pnl_position', 'action','turnover', 'spot', 'spot_value', 'perp_value'])
        df['cash_acc'] = df['cash_acc'].astype('int')
        df['equity'] = df['equity'].astype('int')
        df['funding_payment'] = df['funding_payment'].astype('int')
        df['pnl_position'] = df['pnl_position'].astype('int')

        df['equity_adj'] = df[df['position_open'] == True]['cash_acc'] #we adjust the equity by incorporating negative balances from the cash acc
        df['equity_adj'] = df['equity_adj'].fillna(0) #if the position has been closed, the equity is automatically updated; if its open, there might be some margin balance
        df['equity_adj'] = df['equity_adj'] + df['equity']
        df['equity_adj'] = df['equity_adj'].astype('int')

        df['return'] = 0.0
        df['return'].iloc[0] = (df['equity_adj'].iloc[0]/notional) - 1
        df['return'][1:] = (df['equity_adj'][1:].values/df['equity_adj'][0:-1].values) - 1
        
        df['return_price'] = 0
        df['return_price'].iloc[0] = (df['equity_adj'].iloc[0] - df['funding_payment'].iloc[0]) / notional - 1
        df['return_price'][1:] = ((df['equity_adj'][1:].values - df['funding_payment'][1:].values) / df['equity_adj'][0:-1].values) - 1
    
        df['return_fund'] = 0
        df['return_fund'].iloc[0] = (df['funding_payment'].iloc[0]) / notional
        df['return_fund'][1:] = (df['funding_payment'][1:].values / df['equity_adj'][0:-1].values)

        df_all.append(df)

    dff = []
    for i in range(5):
        dff.append(df_all[i])
    if not consider_funding_rate:
        pd.to_pickle(dff, f'validate_output/output_one_{n_fee}_{q}.pkl')
    else:
        pd.to_pickle(dff, f'validate_output/output_fund_{n_fee}_{q}.pkl')
    
    return dff

Overwriting run_2thre_one.py


In [9]:
for n_fee in [1, 2, 3, 4]:
    with open(f'run_2thre_{n_fee}_one.sbatch', 'w') as fb:
        fb.write(f"""#!/bin/bash

#SBATCH --partition=standard
#SBATCH --account=pi-dachxiu
#SBATCH --job-name=data_pre
#SBATCH --output=JOBLOG/Job_%A_%a.txt
#SBATCH --ntasks=1
#SBATCH --time=7-00:00:00
#SBATCH --mem-per-cpu=4G
#SBATCH --array=1-210

module load anaconda/2021.05
python -c "import run_2thre_one; run_2thre_one.run_2thre_one(${{SLURM_ARRAY_TASK_ID}}, {n_fee})"
""")

In [13]:
import subprocess 
subprocess.run(['sbatch', 'run_2thre_4_one.sbatch'])

CompletedProcess(args=['sbatch', 'run_2thre_4_one.sbatch'], returncode=0)

In [14]:
###########################################################
# select best performing models using OOS validation
###########################################################
import pandas as pd
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')

curr_ls = ['btc', 'eth', 'bnb', 'doge', 'ada']

for n_fee in [4,3,2,1]:
    sum_df_all = pd.DataFrame()
    q_ls_all = []
    if n_fee == 4:
        print('########################################')
        print('Transaction cost: medium-high')
        print('########################################')
    if n_fee == 3:
        print('########################################')
        print('Transaction cost: medium-low')
        print('########################################')
    if n_fee == 2:
        print('########################################')
        print('Transaction cost: low')
        print('########################################')
    if n_fee == 1:
        print('########################################')
        print('Transaction cost: Zero')
        print('########################################')
    for curr_i in range(5):

        f = glob.glob(f'validate_output/output_one_{n_fee}_*')

        s = pd.read_pickle(f[0])
        N = 210

        ret_all = []
        for i in range(5):
            ret_all.append(np.zeros((len(s[i]), N)))

        for q in range(1, N + 1):
            s = pd.read_pickle(f'validate_output/output_one_{n_fee}_{q}.pkl')
            for i in range(5):
                ret_all[i][:, q - 1] = s[i]['return']

        f = glob.glob(f'validate_output/output_one_{n_fee}_*')
        s = pd.read_pickle(f[0])

        s0 = s[curr_i].drop(columns = ['return'])
        s0['timestamp'] = pd.to_datetime(s0['timestamp'])
        s0['yyyymm'] = s0['timestamp'].apply(lambda x: x.year*100 + x.month)

        for n in range(N):
            s0[f'q{n}'] = ret_all[curr_i][:, n]

        ym = list(set(s0['yyyymm']))
        ym.sort()
        
        def get_adj_sr(x):
            return x[x!=0].mean() / x[x!=0].std() * np.sqrt(len(x[x!=0]) / len(x))
        
        def get_adj_mean(x):
            return x[x!=0].mean() * len(x[x!=0]) / len(x)
        
        def get_adj_std(x):
            return x[x!=0].std() * np.sqrt(len(x[x!=0]) / len(x))

        def get_best_q(df_):
            df_ = df_.iloc[:, -210:]
            v = df_.values
            s = np.apply_along_axis(get_adj_sr, 0, v)
            q = np.argmax(s)
            return q

        rolling_month = 6

        q_ls = []
        for i, ym_i in enumerate(ym[rolling_month:]):
            df_ = s0[(s0['yyyymm'] < ym_i) & (s0['yyyymm']>= ym[i])]
            q = get_best_q(df_)
            q_ls.append(q)

        q_df = pd.DataFrame()
        q_df['yyyymm'] = ym[rolling_month:]
        q_df['q'] = q_ls

        s0 = pd.merge(s0, q_df, on = 'yyyymm')
        def get_ret(x):
            return x.loc[f'q{x.q}']

        s0['ret'] = s0.apply(get_ret, axis = 1)
        s0 = s0[['timestamp', 'ret']]
        s0['year'] = s0['timestamp'].apply(lambda x: x.year)

        sum_df = s0.groupby('year').mean()
        sum_df.columns = ['ann_return']
        sum_df['N'] = s0.groupby('year').count()['ret']
        sum_df['prop_active'] = s0[s0['ret'] != 0].groupby('year').count()['ret'] / s0.groupby('year').count()['ret']
        sr_ls = []
        mean_ls = []
        std_ls = []
        for yr in list(set(s0.year)):
            s_ = s0[s0['year'] == yr]
            sr_ls.append(get_adj_sr(s_['ret'])*np.sqrt(24*365))
            mean_ls.append(get_adj_mean(s_['ret'])*24*365)
            std_ls.append(get_adj_std(s_['ret'])*np.sqrt(24*365))
        sum_df['ann_return'] = mean_ls
        sum_df['ann_std'] = std_ls
        sum_df['ann_sharpe'] = sr_ls
        sum_df = sum_df[['N', 'prop_active', 'ann_return', 'ann_std', 'ann_sharpe']]
        sum_df.loc['mean', 'N'] = len(s0)
        sum_df.loc['mean', 'prop_active'] = len(s0[s0['ret'] != 0]) / len(s0)
        sum_df.loc['mean', 'ann_return'] = get_adj_mean(s0['ret'])*24*365
        sum_df.loc['mean', 'ann_std'] = get_adj_std(s0['ret'])*np.sqrt(24*365)
        sum_df.loc['mean', 'ann_sharpe'] = get_adj_sr(s0['ret'])*np.sqrt(24*365)
        sum_df['crypto'] = curr_ls[curr_i]
        sum_df = sum_df.reset_index()
        sum_df_all = sum_df_all.append(sum_df)
        q_ls_all.append(q_ls)
    sum_df_all.set_index(['crypto', 'year']).to_pickle(f'tables/results_one_{n_fee}_v1.pkl')

########################################
Transaction cost: medium-high
########################################
########################################
Transaction cost: medium-low
########################################
########################################
Transaction cost: low
########################################
########################################
Transaction cost: Zero
########################################


In [15]:
###################################################
# Convert results to prettier table
###################################################
for n_fee in [4,3,2,1]:
    df_final = pd.DataFrame(columns = ['crypto', 'stat', '2020', '2021', '2022', 'all'])

    s = pd.read_pickle(f'tables/results_one_{n_fee}_v1.pkl')
    s1 = s.loc['btc']
    df_final['stat'] = s1.columns
    df_final['crypto'] = 'btc'
    df_final.iloc[:, 2:] = s1.values.T

    for crypto in ['eth', 'bnb', 'doge', 'ada']:
        s1 = s.loc[crypto]
        df_ = pd.DataFrame(columns = ['crypto', 'stat', '2020', '2021', '2022', 'all'])
        df_['stat'] = s1.columns
        df_['crypto'] = crypto
        if crypto == 'doge':
            df_.iloc[:, 3:] = s1.values.T
        else:
            df_.iloc[:, 2:] = s1.values.T
        df_final = df_final.append(df_)

    df_final.set_index(['crypto', 'stat']).to_pickle(f'tables/results_one_{n_fee}.pkl')

In [16]:
####################################################################
# decompose the return into the part due to return convergence 
# and the part due to funding rate payment
####################################################################
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from scipy.stats.mstats import winsorize
from random import seed
from random import random
import seaborn as sns
import warnings
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

n_fee = 1
df_all = []

def load_data(load):
    spot = pd.read_csv(f'data/spot_{load}_usdt_1h.csv', parse_dates=['timestamp'])
    perp = pd.read_csv(f'data/perp_{load}_usdt_1h.csv', parse_dates=['timestamp'])
    perp.sort_values(by = 'epoch', ascending=True, inplace=True)
    spot.sort_values(by = 'epoch', ascending=True, inplace=True)
    perp = perp.reset_index(drop=True)
    spot = spot.reset_index(drop=True)
    funding_rate = pd.read_csv(f'data/rate_{load}_usdt.csv', parse_dates=['timestamp'])
    return spot, perp, funding_rate

def create_panel(spot, perp, funding_rate):
    panel = perp[['epoch', 'timestamp', 'close', 'usd_volume']]
    panel['day'] = panel['timestamp'].dt.round('D')
    panel = panel.rename(columns={'close': 'perp', 'usd_volume': 'perp_usd_volume'})
    panel = pd.merge(panel, spot[['epoch', 'close', 'usd_volume']], on='epoch', how='left')
    panel = panel.rename(columns={'close': 'spot', 'usd_volume': 'spot_usd_volume'})
    panel = pd.merge(panel, funding_rate[['epoch', 'funding_rate']], on='epoch', how='left')
    panel.sort_values(by = 'epoch', ascending=True, inplace=True)
    panel = panel.reset_index(drop=True)
    return panel

currencies = ['btc', 'eth', 'bnb', 'doge', 'ada']
panel_all = []
for i in range (0,len(currencies)):
    spot, perp, funding_rate = load_data(currencies[i])
    panel_temp = create_panel(spot, perp, funding_rate)
    panel_all.append(panel_temp)
for i in range (0,len(currencies)):
    T = 1/(3*365)
    panel_all[i]['cip'] = -(1/T)*(np.log(panel_all[i]['perp']) - np.log(panel_all[i]['spot'])) 
    mean = np.round(panel_all[i]['cip'].mean(),4)
    days = (panel_all[i]['day'].iloc[-1] - panel_all[i]['day'].iloc[0]).days
    
cutoff = '2022-11-14 00:00:00'

eth_start = '2019-11-27 07:00:00'
btc_start = '2019-09-10 08:00:00'
doge_start = '2020-07-10 09:00:00'
bnb_start = '2020-02-10 08:00:00'
ada_start = '2020-01-31 08:00:00'

for i in range (0,len(currencies)):
    panel_all[i] = panel_all[i][panel_all[i]['timestamp'] <= cutoff] #cutoff, such that all data is equally long

for i in range (0,len(currencies)):

    panel_all[i]['spot'] = panel_all[i]['spot'].fillna(method='ffill')
    panel_all[i]['spot_usd_volume'] = panel_all[i]['spot_usd_volume'].fillna(method='ffill')
    panel_all[i]['funding_rate'] = panel_all[i]['funding_rate'].fillna(0)
    panel_all[i]['iusdt'] = 0
    #panel_all[i] = panel_all[i].dropna()
    panel_all[i].sort_values(by = 'epoch', ascending=True, inplace=True)
    panel_all[i] = panel_all[i].reset_index(drop=True)

for curr_i in range(5):
    f = glob.glob(f'validate_output/output_one_{n_fee}_*')
    
    s = pd.read_pickle(f[0])
    N = 210
    
    def agg_ret(x):
        f = glob.glob(f'validate_output/output_one_{n_fee}_*')
        s = pd.read_pickle(f[0])
        ret_all = []
        for i in range(5):
            ret_all.append(np.zeros((len(s[i]), N)))
        for q in range(1, N + 1):
            s = pd.read_pickle(f'validate_output/output_one_{n_fee}_{q}.pkl')
            for i in range(5):
                ret_all[i][:, q - 1] = s[i][x] 
        return ret_all
    
    ret_all = agg_ret('return')
    ret_all_price = agg_ret('return_price')
    ret_all_fund = agg_ret('return_fund')
            
    f = glob.glob(f'validate_output/output_one_{n_fee}_*')
    s = pd.read_pickle(f[0])

    s0 = s[curr_i].drop(columns = ['return'])
    s0['timestamp'] = pd.to_datetime(s0['timestamp'])
    s0['yyyymm'] = s0['timestamp'].apply(lambda x: x.year*100 + x.month)

    for n in range(N):
        s0[f'q{n}'] = ret_all[curr_i][:, n]
        s0[f'q{n}_price'] = ret_all_price[curr_i][:, n]
        s0[f'q{n}_fund'] = ret_all_fund[curr_i][:, n]
    ym = list(set(s0['yyyymm']))
    ym.sort()
    
    def get_best_q(df_):
        col_lst = []
        for q in range(N):
            col_lst.append(f'q{q}')
        df_ = df_[col_lst]
        v = df_.values
        s = v.mean(axis = 0) / v.std(axis = 0)
        q = np.argmax(s)
        return q
    
    rolling_month = 6

    q_ls = []
    for i, ym_i in enumerate(ym[rolling_month:]):
        df_ = s0[(s0['yyyymm'] < ym_i) & (s0['yyyymm']>= ym[i])]
        q = get_best_q(df_)
        q_ls.append(q)
        
    q_df = pd.DataFrame()
    q_df['yyyymm'] = ym[rolling_month:]
    q_df['q'] = q_ls
    
    s0 = pd.merge(s0, q_df, on = 'yyyymm')
    def get_ret(x):
        return x.loc[f'q{x.q}']
    def get_ret_price(x):
        return x.loc[f'q{x.q}_price']
    def get_ret_fund(x):
        return x.loc[f'q{x.q}_fund']

    s0['return'] = s0.apply(get_ret, axis = 1)
    s0['return_price'] = s0.apply(get_ret_price, axis = 1)
    s0['return_fund'] = s0.apply(get_ret_fund, axis = 1)
    s0['year'] = s0['timestamp'].apply(lambda x: x.year)
    
    return_lst = []
    price_lst = []
    fund_lst = []
    for n in range(N):
        return_lst.append(f'q{n}')
        price_lst.append(f'q{n}_price')
        fund_lst.append(f'q{n}_fund')
    s0 = s0.drop(columns = return_lst + price_lst + fund_lst).reset_index(drop = True)
    df_all.append(s0)

df_all[3] = df_all[3].append(df_all[0][df_all[0]['year'] == 2020]).sort_values(by = 'timestamp').reset_index(drop = True)
df_all[3].loc[df_all[3]['year'] == 2020, 'return'] = 0
df_all[3].loc[df_all[3]['year'] == 2020, 'return_price'] = 0
df_all[3].loc[df_all[3]['year'] == 2020, 'return_fund'] = 0

#we calculate the statistics mean, std for each year
return_all = []
return_ann_all = []
return_std_all = []
return_ann_std_all = []
risk_free_all = []
trades_all = []
turnover_all = []
fee_all = []
return_price_ann_all = []
return_fund_ann_all = []

for i in range(0, len(currencies)):
    return_temp = []
    return_ann_temp = []
    return_std_temp = []
    return_ann_std_temp = []
    risk_free_temp = []
    trades_temp = []
    turnover_temp = []
    fee_temp = []
    return_price_ann_temp = []
    return_fund_ann_temp = []
    
    if df_all[i]['timestamp'].dt.year.iloc[0] != 2019: #if timeseries starts after 2019
        return_temp.append(0)
        return_ann_temp.append(0)
        return_std_temp.append(0)
        return_ann_std_temp.append(0)
        risk_free_temp.append(0)
        fee_temp.append(0)
        trades_temp.append(0)
        turnover_temp.append(0)
        return_price_ann_temp.append(0)
        return_fund_ann_temp.append(0)
    
    for j in range(df_all[i]['timestamp'].dt.year.iloc[0], df_all[i]['timestamp'].dt.year.iloc[-1]+1):     
        
        return_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['return'].mean()*24)
        return_ann_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['return'].mean()*24*365)
        return_price_ann_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['return_price'].mean()*24*365)
        return_fund_ann_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['return_fund'].mean()*24*365)
        
        return_std_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['return'].std()*np.sqrt(24))
        return_ann_std_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['return'].std()*np.sqrt(24*365))
  
        risk_free_temp.append(panel_all[i][panel_all[i]['timestamp'].dt.year == j]['iusdt'].mean())

        #relative fees paid: fees/(net_equity+fees)
        #fee_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['fee'].sum()/(df_all[i][(df_all[i]['timestamp'].dt.year == j)]['equity_adj'].iloc[-1]+df_all[i][df_all[i]['timestamp'].dt.year == j]['fee'].sum()))
        fee_temp.append(df_all[i][df_all[i]['timestamp'].dt.year == j]['fee'].sum())
        
        trades_temp.append(df_all[i][(df_all[i]['timestamp'].dt.year == j) & (df_all[i]['action'] == True)]['action'].count())
    
        turnover_temp.append(df_all[i][(df_all[i]['timestamp'].dt.year == j)]['turnover'].iloc[-1]-df_all[i][(df_all[i]['timestamp'].dt.year == j)]['turnover'].iloc[0])
    

    return_temp.append(df_all[i]['return'].mean()*24) #mean
    return_all.append(return_temp)

    return_ann_temp.append(df_all[i]['return'].mean()*24*365) #mean
    return_ann_all.append(return_ann_temp)
    
    return_price_ann_temp.append(df_all[i]['return_price'].mean()*24*365)
    return_price_ann_all.append(return_price_ann_temp)
    
    return_fund_ann_temp.append(df_all[i]['return_fund'].mean()*24*365)
    return_fund_ann_all.append(return_fund_ann_temp)
    
    return_std_temp.append(df_all[i]['return'].std()*np.sqrt(24)) #mean
    return_std_all.append(return_std_temp)
    
    return_ann_std_temp.append(df_all[i]['return'].std()*np.sqrt(24*365)) #mean
    return_ann_std_all.append(return_ann_std_temp)
    
    risk_free_temp.append(panel_all[i]['iusdt'].mean()) #mean
    risk_free_all.append(risk_free_temp)
    
    fee_temp.append(sum(fee_temp)/len(fee_temp)) #mean
    fee_all.append(fee_temp)
    
    trades_temp.append(sum(trades_temp)/len(trades_temp)) #mean
    trades_all.append(trades_temp)
    
    turnover_temp.append(sum(turnover_temp)/len(turnover_temp)) #mean
    turnover_all.append(turnover_temp)

#unlisting list for dataframe of results
return_all = [np.round(item,6) for sublist in return_all for item in sublist]
return_std_all = [np.round(item,6) for sublist in return_std_all for item in sublist]
return_ann_all = [np.round(item,6) for sublist in return_ann_all for item in sublist]
return_price_ann_all = [np.round(item, 6) for sublist in return_price_ann_all for item in sublist]
return_fund_ann_all = [np.round(item, 6) for sublist in return_fund_ann_all for item in sublist]
return_ann_std_all = [np.round(item,6) for sublist in return_ann_std_all for item in sublist]
risk_free_all = [np.round(item,6) for sublist in risk_free_all for item in sublist]
fee_all = [np.round(item,4) for sublist in fee_all for item in sublist]
trades_all = [int(item) for sublist in trades_all for item in sublist]
turnover_all = [int(item/1e6) for sublist in turnover_all for item in sublist]
risk_free_all_daily = [item/365 for item in risk_free_all]

sharpe = [(i-j)/k for i,j,k in zip(return_all, risk_free_all_daily, return_std_all)]
sharpe = pd.Series(sharpe).fillna(0).tolist()
sharpe = [np.round(item,4) for item in sharpe]

sharpe_ann = [(i-j)/k for i,j,k in zip(return_ann_all, risk_free_all, return_ann_std_all)]
sharpe_ann = pd.Series(sharpe_ann).fillna(0).tolist()
sharpe_ann = [np.round(item,4) for item in sharpe_ann]

def compute_autoregression(inpt, lags, plot):
    #conducting stationarity test
    df_stationarityTest = adfuller(inpt, autolag='AIC')
    if df_stationarityTest[1] > 0.05:
        print('Data is not stationary!')
    if plot:
        from statsmodels.graphics.tsaplots import plot_pacf
        pacf = plot_pacf(inpt, lags=25)
        plt.show()
    train_data = inpt[:len(inpt)-10]
    test_data = inpt[len(inpt)-10:]
    ar_model = AutoReg(train_data, lags=lags).fit()
    if plot:
        print(ar_model.summary())
    return ar_model.params

def calculate_sharpe_adj(q, k, p, sharpe):
    sm = 0
    for t in range(1, len(p)): # p is params of ar(lag), whereas the first, the coefficient, is skipped.
        sm += (q-k)*p[t]
    n = q / np.sqrt(q+2*sm)
    sharpe_adj = sharpe * n
    return sharpe_adj

ar1_params = []
lags = 1 #lags of autocorrelation
periods = 365 #days of the year

for i in range(0, len(currencies)):

    if df_all[i]['timestamp'].dt.year.iloc[0] != 2019: #if timeseries starts after 2019
        ar1_params.append([0,0])

    for j in range(df_all[i]['timestamp'].dt.year.iloc[0], df_all[i]['timestamp'].dt.year.iloc[-1]+1):
        ar1_params.append(compute_autoregression(inpt=df_all[i][df_all[i]['timestamp'].dt.year == j]['return'], lags=lags, plot=False))

    ar1_params.append(compute_autoregression(inpt=df_all[i]['return'], lags=lags, plot=False)) #for all years

sharpe_ar1 = []

for t in range(0, len(ar1_params)):
    p = ar1_params[t]
    sharpe_ar1.append(calculate_sharpe_adj(q=periods, k=lags, p=p, sharpe=sharpe[t].copy()))

iterables = [currencies, ['2019', '2020', '2021', '2022', 'mean']]

index = pd.MultiIndex.from_product(iterables)

results = pd.DataFrame(
    {'daily_return': return_all,
    'daily_std': return_std_all,
    'sharpe': sharpe,
    'ann_return': return_ann_all,
    'ann_std': return_ann_std_all,
    'ann_risk_free': risk_free_all,
    'ann_sharpe': sharpe_ann,
    'ann_sharpe_ar1': sharpe_ar1,
    'fees': fee_all,
    'trades': trades_all,
    'turnover': turnover_all},index = index)

return_ann_all = [np.round(item,4) for item in return_ann_all]
return_price_ann_all = [np.round(item,4) for item in return_price_ann_all]
return_fund_ann_all = [np.round(item,4) for item in return_fund_ann_all]
return_ann_std_all = [np.round(item,4) for item in return_ann_std_all]
sharpe_ann = [np.round(item,2) for item in sharpe_ann]
sharpe_ar1 = [np.round(item,2) for item in sharpe_ar1]

# return_ann_all = ['' if item == 0 else item for item in return_ann_all]
# return_price_ann_all = ['' if item == 0 else item for item in return_price_ann_all]
# return_fund_ann_all = ['' if item == 0 else item for item in return_fund_ann_all]
# return_ann_std_all = ['' if item == 0 else item for item in return_ann_std_all]
# sharpe_ann = ['' if item == 0 else item for item in sharpe_ann]
# sharpe_ar1 = ['' if item == 0 else item for item in sharpe_ar1]

return_ann_std_all_pretty = ['' if item == '' else '(' + str(item)+ ')' for item in return_ann_std_all]

iterables = [currencies, ['ann_return', 'ann_std', 'ann_sharpe', 'ann_sharpe_ar1', 'return_price', 'return_fund']]

index = pd.MultiIndex.from_product(iterables)

results_pretty = pd.DataFrame(
    {'2019': [item for sublist in list(zip(return_ann_all[0::5], return_ann_std_all_pretty[0::5], sharpe_ann[0::5], sharpe_ar1[0::5], return_price_ann_all[0::5], return_fund_ann_all[0::5])) for item in sublist],
    '2020': [item for sublist in list(zip(return_ann_all[1::5], return_ann_std_all_pretty[1::5], sharpe_ann[1::5], sharpe_ar1[1::5], return_price_ann_all[1::5], return_fund_ann_all[1::5])) for item in sublist],
    '2021': [item for sublist in list(zip(return_ann_all[2::5], return_ann_std_all_pretty[2::5], sharpe_ann[2::5], sharpe_ar1[2::5], return_price_ann_all[2::5], return_fund_ann_all[2::5])) for item in sublist],
    '2022': [item for sublist in list(zip(return_ann_all[3::5], return_ann_std_all_pretty[3::5], sharpe_ann[3::5], sharpe_ar1[3::5], return_price_ann_all[3::5], return_fund_ann_all[3::5])) for item in sublist], 
    'all': [item for sublist in list(zip(return_ann_all[4::5], return_ann_std_all_pretty[4::5], sharpe_ann[4::5], sharpe_ar1[4::5], return_price_ann_all[4::5], return_fund_ann_all[4::5])) for item in sublist]},index = index)
results_pretty = results_pretty.dropna(axis=0) #drop years without data
results_pretty = results_pretty.drop(columns = ['2019'])
results_pretty.to_pickle(f'tables/results_one_{n_fee}_decompose.pkl')

results_pretty

2020      2021      2022       all
btc  ann_return        0.0963    0.2189    0.0029    0.1116
     ann_std         (0.0137)  (0.0294)  (0.0018)  (0.0195)
     ann_sharpe          7.02      7.44       1.6      5.73
     ann_sharpe_ar1      7.68      9.06      1.59      6.81
     return_price      0.0632    0.1169    0.0029    0.0637
     return_fund       0.0331     0.102       0.0    0.0479
eth  ann_return        0.1738    0.3049    0.0145     0.171
     ann_std         (0.0297)   (0.033)  (0.0076)  (0.0261)
     ann_sharpe          5.85      9.23      1.92      6.54
     ann_sharpe_ar1      5.88     11.51      2.19      7.44
     return_price      0.1004    0.1853    0.0153    0.1047
     return_fund       0.0735    0.1196   -0.0008    0.0662
bnb  ann_return        0.3868    0.4377    0.0722    0.2896
     ann_std         (0.0293)  (0.0383)  (0.0058)  (0.0285)
     ann_sharpe         13.19     11.43     12.46     10.15
     ann_sharpe_ar1     17.34      15.5     13.74     13.49
     return_price      0.3165    0.3492    0.0727    0.2382
     return_fund       0.0704    0.0885   -0.0005    0.0514
doge ann_return           0.0    0.6685    0.1346    0.2901
     ann_std            (0.0)  (0.0633)  (0.0096)   (0.039)
     ann_sharpe           0.0     10.57     14.04      7.45
     ann_sharpe_ar1       0.0     13.15      15.0      9.13
     return_price         0.0    0.5037    0.1328    0.2287
     return_fund          0.0    0.1648    0.0018    0.0614
ada  ann_return        0.4079    0.4605    0.1363    0.3307
     ann_std         (0.0305)  (0.0448)  (0.0091)  (0.0328)
     ann_sharpe         13.39     10.27     14.93     10.07
     ann_sharpe_ar1     16.94     12.29     15.29     12.07
     return_price      0.2831    0.3174    0.1346    0.2432
     return_fund       0.1248    0.1431    0.0018    0.0875